In [1]:
import json
import requests
import re
from IPython import display
from datetime import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [23]:
FIREBASE_PRIVATE_KEY = r'C:\Users\johnp\Documents\ESportsAnalytics\esportsanalytics-3b04cf0c1d47.json'
BASE_URL = 'https://api.opendota.com/api/'
TEAMS = 'teams'
HEROES = 'heroes'
PROPLAYERS = 'proPlayers'
MATCHES = 'matches'

In [3]:
# Connect to Firebase and return a database instance
def connecttofirebase(PATH):
    
    # Check if the app is already initialised, if not then connect
    if not firebase_admin._apps:
        cred = credentials.Certificate(FIREBASE_PRIVATE_KEY)
        firebase_admin.initialize_app(cred)
    
    # Get an instance of the project database
    db = firestore.client()
    
    return db

In [4]:
db = connecttofirebase(FIREBASE_PRIVATE_KEY)

In [5]:
db.project

'esportsanalytics'

In [64]:
TODO: Simplify codes to basic ones e.g. update_document(), check

a0
b1
c2


In [66]:
# Do these if data doesn't exist yet or is not up to date
response = requests.get(BASE_URL + PROPLAYERS)

try:
    pro_players_data_json = json.loads(response.content.decode('utf-8'))

except requests.exceptions.Timeout:
    print('request is taking too long to complete, possible timeout, getting old data from Firebase')

except requests.exceptions.RequestException as e:
    raise SystemExit(e)

else:
    
    pro_players_split = [
        {u'all_pro_players_info_1' : pro_players_data_json[0:1000], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},
        {u'all_pro_players_info_2' : pro_players_data_json[1000:2000], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},
        {u'all_pro_players_info_3' : pro_players_data_json[2000:], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},
        
    ]

    for index, data in enumerate(pro_players_split):
        pro_players_ref = db.collection(u'dota').document(u'all_pro_players_'+str(index)).set(data)
    
    print("All Teams data uploaded to Firebase")
    

All Teams data uploaded to Firebase


In [62]:
# Save the JSON as a file in the working directory
with open('pro_players_data.json', 'w', encoding='utf-8') as f:
    json.dump(pro_players_data_json[0:1000], f, ensure_ascii=False, indent=4)

In [25]:
def get_all_pro_players():
    # check if data already exist in Firebase
    doc_ref = db.collection(u'dota').document(u'all_pro_players')
    doc = doc_ref.get()
    
    data_exist = doc.exists
    
    # If it exists, check the age of the data
    if(data_exist):
        print("All Pro Players already exist in Fireabse")
        
        datetime_upload_str = doc.to_dict()['date_time_upload']
        datetime_upload = datetime.strptime(datetime_upload_str, "%m/%d/%Y, %H:%M:%S")
        
        # subtracting two datetime objects result in a datetime.timedelta object
        data_age_timedelta = datetime.now() - datetime_upload
        data_age_in_days = data_age_timedelta.total_seconds() / 86400
        
        print("All Pro Players Info is:", data_age_in_days, "days old")
        
        # Decide when to do a proper data sync / update depending on data age
        if (data_age_in_days > 1):
            print("Data is more than 1 day old, resyncing from OpenDota......")
            
            # Do these if data doesn't exist yet or is not up to date
            response = requests.get(BASE_URL + PROPLAYERS)

            try:
                pro_players_data_json = json.loads(response.content.decode('utf-8'))

            except requests.exceptions.Timeout:
                print('request is taking too long to complete, possible timeout, getting old data from Firebase')

            except requests.exceptions.RequestException as e:
                raise SystemExit(e)
            
            else:
                 pro_players_split = [
                    {u'all_pro_players_info_1' : pro_players_data_json[0:1000], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},
                    {u'all_pro_players_info_2' : pro_players_data_json[1000:2000], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},
                    {u'all_pro_players_info_3' : pro_players_data_json[2000:], u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),},

                ]

                for index, data in enumerate(pro_players_split):
                    pro_players_ref = db.collection(u'dota').document(u'all_pro_players_'+str(index)).set(data)
                
    
        # Get the data from Firebase
        pro_players_data_json = doc.to_dict()['all_heroes_info']
        
        return pro_players_data_json

In [7]:
# Get information on all professional dota teams
def get_all_heroes_info():
    
    # check if data already exist in Firebase
    doc_ref = db.collection(u'dota').document(u'all_heroes')
    doc = doc_ref.get()
    
    data_exist = doc.exists
    
    # If it exists, check the age of the data
    if(data_exist):
        print("All Heroes already exist in Fireabse")
        
        datetime_upload_str = doc.to_dict()['date_time_upload']
        datetime_upload = datetime.strptime(datetime_upload_str, "%m/%d/%Y, %H:%M:%S")
        
        # subtracting two datetime objects result in a datetime.timedelta object
        data_age_timedelta = datetime.now() - datetime_upload
        data_age_in_days = data_age_timedelta.total_seconds() / 86400
        
        print("All Heroes Info is:", data_age_in_days, "days old")
        
        # Decide when to do a proper data sync / update depending on data age
        if (data_age_in_days > 1):
            print("Data is more than 1 day old, resyncing from OpenDota......")
            
            # Do these if data doesn't exist yet or is not up to date
            response = requests.get(BASE_URL + HEROES)

            try:
                heroes_data_json = json.loads(response.content.decode('utf-8'))

            except requests.exceptions.Timeout:
                print('request is taking too long to complete, possible timeout, getting old data from Firebase')

            except requests.exceptions.RequestException as e:
                raise SystemExit(e)
            
            else:
                # upload the team_data_json in Firebase if there are no errors
                data = {
                    u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
                    u'all_heroes_info' : heroes_data_json
                }

                db.collection(u'dota').document(u'all_heroes').set(data)
                print("All Teams data uploaded to Firebase")
                
    
        # Get the data from Firebase
        heroes_data_json = doc.to_dict()['all_heroes_info']
        return heroes_data_json

In [8]:
# Get information on all professional dota teams
def get_all_teams_info():
    
    # check if data already exist in Firebase
    doc_ref = db.collection(u'dota').document(u'all_teams')
    doc = doc_ref.get()
    
    data_exist = doc.exists
    
    # If it exists, check the age of the data
    if(data_exist):
        print("All Teams already exist in Fireabse")
        
        datetime_upload_str = doc.to_dict()['date_time_upload']
        datetime_upload = datetime.strptime(datetime_upload_str, "%m/%d/%Y, %H:%M:%S")
        
        # subtracting two datetime objects result in a datetime.timedelta object
        data_age_timedelta = datetime.now() - datetime_upload
        data_age_in_days = data_age_timedelta.total_seconds() / 86400
        
        print("All Teams Info is:", data_age_in_days, "days old")
        
        # Decide when to do a proper data sync / update depending on data age
        if (data_age_in_days > 1):
            print("Data is more than 1 day old, resyncing from OpenDota......")
            
            # Do these if data doesn't exist yet or is not up to date
            response = requests.get(BASE_URL + TEAMS)

            try:
                team_data_json = json.loads(response.content.decode('utf-8'))

            except requests.exceptions.Timeout:
                print('request is taking too long to complete, possible timeout, getting old data from Firebase')

            except requests.exceptions.RequestException as e:
                raise SystemExit(e)
            
            else:
                # upload the team_data_json in Firebase if there are no errors
                data = {
                    u'date_time_upload' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
                    u'all_teams_info' : team_data_json
                }

                db.collection(u'dota').document(u'all_teams').set(data)
                print("All Teams data uploaded to Firebase")
                
    
        # Get the data from Firebase
        team_data_json = doc.to_dict()['all_teams_info']
        return team_data_json

In [9]:
def get_all_players():
    return 0

In [10]:
team_data_json = get_all_teams_info()

All Teams already exist in Fireabse
All Teams Info is: 0.9682175185185186 days old


In [14]:
def search_regex(search_string, team_from_list):
    try:
        x = re.search(search_string, team_from_list, re.IGNORECASE).group()
    except Exception:
        return False
    return x

In [15]:
def search_team(team_name):
    team_filter =  filter(lambda y: search_regex(team_name, y['name']), team_data_json)
    return [team_info for team_info in team_filter]

In [16]:
x = search_team('team')

In [17]:
heroes = get_all_heroes_info()

All Heroes already exist in Fireabse
All Heroes Info is: 0.9533700312152777 days old


In [21]:
pro_players = get_all_pro_players()

In [32]:
with open('all_teams_data.json', 'w', encoding='utf-8') as f:
    json.dump(team_data_json, f, ensure_ascii=False, indent=4)